In [1]:
import deepface
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import random
from path import Path
import csv
import tensorflow as tf
from sklearn.metrics import f1_score, confusion_matrix
from deepface import DeepFace
from deepface.modules import verification, representation, detection, preprocessing
from deepface.models.FacialRecognition import FacialRecognition
from deepface.commons import logger as log
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
#from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from keras.preprocessing.image import load_img
import onnxruntime
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image
from typing import List, Union, Optional, Dict, Any
import time
from tqdm import tqdm

print("hello world")

C:\Users\Admin\anaconda3\envs\faceMatch\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(



hello world


In [2]:
def load_images(folder_path):
    image_folder = folder_path
    images= [img for img in os.listdir(image_folder)]
    
def get_image_files(folder_path):
  image_files = []
  for dirpath, dirnames, filenames in os.walk(folder_path):
    for filename in filenames:
      # Checking if filename ends with a common image extension
      if filename.lower().endswith((".jpg", ".jpeg", ".png", ".gif")):
        image_files.append(os.path.join(dirpath, filename))
  return image_files

# Example usage
folder_path = r"C:\Users\Admin\OneDrive\Desktop\shivani\geekroom\Faces"
image_files = get_image_files(folder_path)

for images in image_files:
  print(images)  # Print the full path to each image file

C:\Users\Admin\OneDrive\Desktop\shivani\geekroom\Faces\Akshay Kumar\Akshay Kumar_0.jpg
C:\Users\Admin\OneDrive\Desktop\shivani\geekroom\Faces\Akshay Kumar\Akshay Kumar_1.jpg
C:\Users\Admin\OneDrive\Desktop\shivani\geekroom\Faces\Akshay Kumar\Akshay Kumar_10.jpg
C:\Users\Admin\OneDrive\Desktop\shivani\geekroom\Faces\Akshay Kumar\Akshay Kumar_11.jpg
C:\Users\Admin\OneDrive\Desktop\shivani\geekroom\Faces\Akshay Kumar\Akshay Kumar_12.jpg
C:\Users\Admin\OneDrive\Desktop\shivani\geekroom\Faces\Akshay Kumar\Akshay Kumar_13.jpg
C:\Users\Admin\OneDrive\Desktop\shivani\geekroom\Faces\Akshay Kumar\Akshay Kumar_14.jpg
C:\Users\Admin\OneDrive\Desktop\shivani\geekroom\Faces\Akshay Kumar\Akshay Kumar_15.jpg
C:\Users\Admin\OneDrive\Desktop\shivani\geekroom\Faces\Akshay Kumar\Akshay Kumar_16.jpg
C:\Users\Admin\OneDrive\Desktop\shivani\geekroom\Faces\Akshay Kumar\Akshay Kumar_17.jpg
C:\Users\Admin\OneDrive\Desktop\shivani\geekroom\Faces\Akshay Kumar\Akshay Kumar_18.jpg
C:\Users\Admin\OneDrive\Desktop\sh

In [3]:
#creating the required dataset
mainList = []
person_groups = {}
counter=0
counts = 0
def get_person_name(image_path):
    """Extract the person's name from the image file name."""
    return os.path.basename(image_path).split('_')[0]

def select_image_pairs(image_paths):
    
    # Grouping images by person name
    
    for img_path in image_paths:
        person_name = get_person_name(img_path)
        if person_name not in person_groups:
            person_groups[person_name] = []
        person_groups[person_name].append(img_path)

    # Randomly selecting image pairs
    for person_name, images in person_groups.items():
        if len(images) > 1:
            first_image = random.choice(images)
            second_image = random.choice([img for img in images if img != first_image])
            mainList.append([first_image,second_image,1])  # Label 1 for the same person
            global counter
            counter += 1

folder = r'C:\Users\Admin\OneDrive\Desktop\shivani\geekroom\Faces'


for i in range(0,250):
    list = []
    for j in range(0,2): 
        if images in image_files:
            selected_image = random.choice(image_files)
            list.append(selected_image)
    if list[0].split('_') == list[1].split('_'):
        list.append(1)
        counts += 1
    else:
        list.append(0)
    mainList.append(list)
for j in range(0,8):
     select_image_pairs(image_files)
random.shuffle(mainList)
print(counter + counts)
#print(f"CSV file '{csv_filename}' created successfully.")

248


In [4]:
#writing into a csv file, the dataset that we just created
import csv

# Defining the CSV file name
csv_filename = 'image_pairs.csv'


with open(csv_filename, mode = 'w', newline = '') as file:
    writer = csv.writer(file)
    writer.writerow(['Image 1', 'Image 2', 'Label'])
    for row in mainList:
        writer.writerow(row)
print(f"CSV file {csv_filename} created successfully")

CSV file image_pairs.csv created successfully


In [5]:
# Loading VGG(visual geometry group)-Face model (Using VGG16 (16 CNN layers) pre-trained on ImageNet as an example)
#not specific to the human face, vgg16 has been trained on many datasets; vggFace has been trained on dataset specific to human face
#Here, we are exploring the method of image processing without the use of deepface library
base_model = VGG16(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)

def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)
    return img_data

def extract_features(img_path):
    img_data = load_and_preprocess_image(img_path)
    features = model.predict(img_data)
    return features

def cosine_similarity(features1, features2):
    cos_sim = np.dot(features1, features2.T) / (np.linalg.norm(features1) * np.linalg.norm(features2))  #x.y/|x||y|
    return cos_sim

# Loading the  CSV file - previously defined
csv_filename = 'image_pairs.csv'
data = pd.read_csv(csv_filename)

# Initialize lists to store results
similarities = []
actual_labels = []

# Process each pair of images- the variable index has no effect on the processing
for index, row in data.iterrows():
    img_path1 = row['Image 1']
    img_path2 = row['Image 2']
    label = row['Label']
    
    features1 = extract_features(img_path1)
    features2 = extract_features(img_path2)
    
    similarity = cosine_similarity(features1, features2)
    similarities.append(similarity)
    actual_labels.append(label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 730ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 633ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 839ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 639ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 717ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 640ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 760ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 580ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 515ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 617ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 687ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 527ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 594ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 568ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 490ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 489ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 520ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 518ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 514ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 478ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 512ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 498ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 581ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 572ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 

In [6]:
#calculating optimum threshold +f1score for vgg model
thresholds = np.arange(0.0, 1.0, 0.01)
best_threshold = 0.0
best_f1 = 0.0 #this is the worst case; best case is 1, we replace best_f1 with the greater value below

for threshold in thresholds:
    predictions = [1 if sim >= threshold else 0 for sim in similarities]
    f1 = f1_score(actual_labels, predictions)
    
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

print("the optimum threshold value is: ", best_threshold)
print("The best f1 value is: ", best_f1)

best_predictions = [1 if sim>= best_threshold else 0 for sim in similarities]
conf_matrix = confusion_matrix(actual_labels, best_predictions)
print(conf_matrix)


# Evaluate results
#for pred, actual in predictions, actual_labels
correct_predictions = sum([pred == actual for pred, actual in zip(best_predictions, actual_labels)])
total_predictions = len(predictions)
accuracy = correct_predictions / total_predictions
precision = conf_matrix[0][0]/(conf_matrix[0][0]+conf_matrix[0][1])
recall = conf_matrix[0][0]/(conf_matrix[0][0]+conf_matrix[1][0])

print(f"Accuracy of vgg face-model is: {accuracy * 100:.2f}%")
print(f"Precision of vgg face-model is: {precision * 100:.2f}%")
print(f"Recall of vgg face-model is: {recall * 100:.2f}%")

the optimum threshold value is:  0.59
The best f1 value is:  0.6989079563182528
[[ 81 169]
 [ 24 224]]
Accuracy of vgg face-model is: 61.24%
Precision of vgg face-model is: 32.40%
Recall of vgg face-model is: 77.14%


In [7]:
csv_filename = 'image_pairs.csv'
data = pd.read_csv(csv_filename)
image1 = []
image2 = []
actual_labels = []
for index, row in data.iterrows():
    image1.append(row["Image 1"])
    image2.append(row["Image 2"])
    actual_labels.append(row["Label"])
print(image1[0])

C:\Users\Admin\OneDrive\Desktop\shivani\geekroom\Faces\Akshay Kumar\Akshay Kumar_19.jpg


In [8]:
#use deepface library for the Facenet model, FaceNet returns 128 dimensions, VGG-Face returns 4096 dimensions
#we are using the extract_faces function here
#extract faces function: used to detect and isolate faces in an image containing multiple faces
print("block start")
extracted_image = [[],[]]
for Image1 in image1:
    extracted_image[0].append(DeepFace.extract_faces(Image1,detector_backend = "opencv", enforce_detection =False,align=True))
for Image2 in image2:
    extracted_image[1].append(DeepFace.extract_faces(Image2,detector_backend = "opencv", enforce_detection =False,align=True))
print(extracted_image[0][0])
print("block over")

block start
[{'face': array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       ...,

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]]), 'facial_area': {'x': -2, 'y': -5, 'w': 167, 'h': 167, 'left_eye': (107, 53), 'righ

In [9]:
#creating faceNet model
faceNetModel = DeepFace.build_model(model_name = "Facenet")
print("model built")

model built


In [10]:
#using model for preprocessing of images
#Here, we are normalizing the pixel intensity
normalized_image1 = []
normalized_image2 = []
for Image1 in image1:
    img = image.load_img(Image1)
    normalized_img = preprocessing.normalize_input(image.img_to_array(img),normalization="Facenet")
    normalized_image1.append(normalized_img)
for Image2 in image2:
    img = image.load_img(Image2)
    normalized_img = preprocessing.normalize_input(image.img_to_array(img),normalization="Facenet")
    normalized_image2.append(normalized_img)
print(np.array(normalized_image1).shape, np.array(normalized_image2).shape)

(498, 160, 160, 3) (498, 160, 160, 3)


In [11]:
#after normalizing the image, we are now resizing them to the standard format
resized_image1 = []
resized_image2 = []
for img1 in normalized_image1:
    resized_image1.append(preprocessing.resize_image(img1, target_size = ([224, 224])))
for img2 in normalized_image2:
    resized_image2.append(preprocessing.resize_image(img2, target_size = ([224, 224])))
print(np.array(resized_image1).shape,np.array(resized_image1).shape)

(498, 1, 224, 224, 3) (498, 1, 224, 224, 3)


In [12]:
#using the normalized images in represent function to create embeddings
embeddings1 = []
embeddings2 = []
for img1 in normalized_image1:
    embedding = DeepFace.represent(img_path = img1, 
                                    model_name = "Facenet", detector_backend="opencv", normalization="Facenet", enforce_detection=False)[0]['embedding']
    
    embeddings1.append([embedding])
for img2 in normalized_image2:
    embedding = DeepFace.represent(img_path = img2, 
                                    model_name = "Facenet", detector_backend="opencv", normalization="Facenet", enforce_detection=False)[0]['embedding']
    
    embeddings2.append([embedding])
print(np.array(embeddings1).shape,np.array(embeddings2).shape)

(498, 1, 128) (498, 1, 128)


In [13]:
'''
#Here, we are creating embeddings using the image paths stored in the lists
# The [0]['embedding'] part extracts the embedding vector from the first dictionary in the list.
embeddings1 = []
embeddings2 = []
for img1 in image1:
    embedding = DeepFace.represent(img_path = img1, 
                                    model_name = "Facenet", detector_backend="opencv", normalization="Facenet", enforce_detection=False)[0]['embedding']
    
    embeddings1.append([embedding])
for img2 in image2:
    embedding = DeepFace.represent(img_path = img2, 
                                    model_name = "Facenet", detector_backend="opencv", normalization="Facenet", enforce_detection=False)[0]['embedding']
    
    embeddings2.append([embedding])
print(np.array(embeddings1).shape,np.array(embeddings2).shape)
'''

(498, 1, 128) (498, 1, 128)


In [14]:
def cosine_similarity(features1, features2):
    cos_sim = np.dot(features1,features2.T) / (np.linalg.norm(features1) * np.linalg.norm(features2))  #x.y/|x||y|
    return cos_sim
similaritiesFaceNet = []

for i in range(0,498):
    similarity = cosine_similarity(np.array(embeddings1[i][0]), np.array(embeddings2[i][0]))
    similaritiesFaceNet.append(similarity)
'''
print(np.array(embeddings2[0]).shape)
'''

'\nprint(np.array(embeddings2[0]).shape)\n'

In [15]:
thresholds = np.arange(0.0, 1.0, 0.01)
best_threshold = 0.0
best_f1 = 0.0

for threshold in thresholds:
    predictions = [1 if sim>= threshold else 0 for sim in similaritiesFaceNet]
    f1_FaceNet = f1_score(actual_labels, predictions)
    if f1_FaceNet > best_f1:
        best_f1 = f1_FaceNet
        best_threshold = threshold


print("the optimum threshold value is: ", best_threshold)
print("The best f1 value is: ", best_f1)

best_predictions = [1 if sim>=best_threshold else 0 for sim in similaritiesFaceNet]
conf_matrix = confusion_matrix(actual_labels, best_predictions)
print(conf_matrix)

the optimum threshold value is:  0.4
The best f1 value is:  0.9475806451612904
[[237  13]
 [ 13 235]]


In [16]:
accuracy = (conf_matrix[0][0] + conf_matrix[1][1])/(conf_matrix[0][0] + conf_matrix[1][1] + conf_matrix[0][1] + conf_matrix[1][0])
precision = (conf_matrix[0][0])/(conf_matrix[0][0] + conf_matrix[0][1])
recall = (conf_matrix[0][0])/(conf_matrix[0][0] + conf_matrix[1][0])
print(f"The accuracy obtained using FaceNet library and deepface functions is: {accuracy*100:.2f}%")
print(f"The precision obtained here is: {precision*100:.2f}%")
print(f"The recall obtained here is: {recall*100:.2f}%")

The accuracy obtained using FaceNet library and deepface functions is: 94.78%
The precision obtained here is: 94.80%
The recall obtained here is: 94.80%


In [ ]:
thresholds = np.arange(0.0, 1.0, 0.01)
best_threshold = 0.0
best_f1 = 0.0


In [42]:
from deepface.modules import streaming, modeling, demography
for i in embeddings1:
    analysed_image = streaming.perform_demography_analysis(enable_face_analysis = True,img = i, faces_coordinates = streaming.grab_facial_areas(i, detector_backend= 'Facenet' ,threshold = 0.5), detected_faces= i)
print(analysed_image[0])

[1.0391725301742554, -0.11941701918840408, -0.8449068665504456, -1.562268614768982, -0.7371090054512024, 2.262735366821289, -2.6971778869628906, -1.6899592876434326, 2.1328089237213135, -0.5523298978805542, 0.8370856046676636, -0.8597925901412964, -2.4810261726379395, 0.9111628532409668, -1.2974910736083984, -0.4495026469230652, -0.20527279376983643, -0.9078401327133179, -0.6670446395874023, -3.13909912109375, -0.1444816142320633, -0.3247825801372528, 0.019064366817474365, -0.904449462890625, -0.10992881655693054, -0.8310995697975159, -1.2179182767868042, 0.1365591585636139, 1.6837804317474365, 1.4040285348892212, -0.5257917642593384, 0.13721896708011627, 0.5343058109283447, 0.15402725338935852, 0.8361476063728333, 2.635413408279419, -1.4716496467590332, 1.037755012512207, -0.23694190382957458, -0.45307883620262146, 2.890408515930176, -1.824161410331726, -1.4424439668655396, -2.0350594520568848, -0.6120890378952026, -1.4060876369476318, 0.2319839894771576, 0.6244909763336182, -1.223256

In [54]:
print(image1[0])
analysis = DeepFace.analyze(img_path=image1[0] ,actions=['age','gender'])
for face in analysis:
  print(face["age"])

C:\Users\Admin\OneDrive\Desktop\shivani\geekroom\Faces\Akshay Kumar\Akshay Kumar_19.jpg


Action: age:   0%|                                                                               | 0/2 [00:03<?, ?it/s]


ValueError: The layer sequential_29 has never been called and thus has no defined input.